In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
train = pd.read_csv('/gdrive/My Drive/Data/trains.csv')
test = pd.read_csv('/gdrive/My Drive/Data/test.csv')

for col in train.columns:
    col_type = train[col].dtypes
    min1 = train[col].min()
    max1 = train[col].max()
    if str(col_type)[:3] == 'int':
        train[col] = train[col].astype(np.int16)
    else:
        if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
            train[col] = train[col].astype(np.float16)
        elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
            train[col] = train[col].astype(np.float32)
        else:
            train[col] = train[col].astype(np.float64)
train = train.sample(frac=1).reset_index(drop=True)
train_X = train.iloc[:,4:]
train_Y = train.iloc[:,0:4]
test_X = test.iloc[:,1:]

In [0]:
from keras import backend as K

def mish(x):
        return x * K.tanh( K.softplus(x))

#K.get_custom_objects().update({'mish': Activation(mish)})

In [3]:
train_set = train_X.iloc[:690000,:], train_Y.iloc[:690000,:]
val_set = train_X.iloc[690000:,:], train_Y.iloc[690000:,:]
print(train_set[0].shape)
print(val_set[0].shape)

(690000, 226)
(120000, 226)


In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras import optimizers
from keras import activations


def build_model(input_s,dropout_rate1,dropout_rate2,dropout_rate3,dropout_rate4,lr):
    model = Sequential()
    model.add(Dense(units=239, activation=mish, input_dim=input_s))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate1))
    model.add(Dense(units=252, activation=mish, input_dim=226))
    model.add(Dense(units=252, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate2))
    model.add(Dense(units=265, activation=mish, input_dim=226))
    model.add(Dense(units=265, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate3))
    model.add(Dense(units=178, activation=mish, input_dim=226))
    model.add(Dense(units=178, activation=mish, input_dim=226))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate4))
    model.add(Dense(units=91, activation=mish, input_dim=226))
    model.add(Dense(units=91, activation=mish, input_dim=226))
    model.add(Dense(units=4, activation='linear'))
    op = optimizers.Nadam(lr = lr , beta_1 = 0.9, beta_2 = 0.999)
    model.compile(loss='mae', optimizer=op, metrics=['mae'])
    return model
# def swish(x) :
#     return x * keras.activations.sigmoid(x)

# def mish(x) :
#     return x * keras.activations.tanh( keras.activations.softplus(x))

Using TensorFlow backend.


In [0]:
def fit_with(verbose, lr, dropout_rate1,dropout_rate2,dropout_rate3,dropout_rate4):
    model = build_model(226, dropout_rate1,dropout_rate2,dropout_rate3,dropout_rate4, lr)
    fit_with = model.fit(train_set[0],train_set[1], epochs=10, batch_size=1000, shuffle=True)
    e_val = model.evaluate(val_set[0],val_set[1],steps=10, verbose=0)
    print('Test loss:', e_val[0])
    print('Test score:', e_val[1])
    return -(e_val[0])

from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, verbose)

In [8]:
new_model = keras.models.load_model('/gdrive/My Drive/ckpt/tuning_254_valloss1.90.hdf5',custom_objects={'mish': mish})
new_model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_221 (Dense)            (None, 239)               54253     
_________________________________________________________________
batch_normalization_89 (Batc (None, 239)               956       
_________________________________________________________________
dropout_89 (Dropout)         (None, 239)               0         
_________________________________________________________________
dense_222 (Dense)            (None, 252)               60480     
_________________________________________________________________
dense_223 (Dense)            (None, 252)               63756     
_________________________________________________________________
batch_normalization_90 (Batc (None, 252)               10

In [6]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=b59bc485c475b7b8b79328b2931dffdc59c4d49af92f8cddc015c80203b550db
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [7]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout_rate1': (1e-8, 0.2), 
           'dropout_rate2': (1e-8, 0.2),
           'dropout_rate3': (1e-8, 0.2),
           'dropout_rate4': (1e-8, 0.2),
           'lr': (1e-4, 1e-2)}

bayes_optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

bayes_optimizer.maximize(init_points=10, n_iter=10)
for i, res in enumerate(bayes_optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(bayes_optimizer.max)

|   iter    |  target   | dropou... | dropou... | dropou... | dropou... |    lr     |
-------------------------------------------------------------------------------------




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Epoch 1/10





690000/690000 [==============================] - 25s 37us/step - loss: 57.1779 - mean_absolute_error: 57.1779
Epoch 2/10
690000/690000 [==============================] - 15s 21us/step - loss: 29.4497 - mean_absolute_error: 29.4497
Epoch 3/10
690000/690000 [==============================] - 15s 22us/step - loss: 15.7362 - mean_absolute_error: 15.7362
Epoch 4/10
690000/690000 [==============================] - 15s 21us/step - loss: 12.5229 - mean_absolute_error: 12.5229
Epoch 5/10
690000/690000 [==============================] - 15s 21us/step - loss: 11.0154 - mean_absolute_error: 11.0154
Epoch 6/10
690000/690000 [==============================] - 15s 21us/step - loss: 10.0890 - mea

In [7]:
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import keras.backend as K
plt.figure(figsize = (13,8))
class LRFinder(Callback):
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.ylim(0, 10)

early_stop = keras.callbacks.EarlyStopping(patience=50, monitor='val_loss')

ckpt_dir = '/gdrive/My Drive/ckpt'
ckpt_path = ckpt_dir + '/tuning_{epoch:02d}_valloss{val_loss:.2f}.hdf5'
ckpt = keras.callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')


model = build_model(226, dropout_rate1=0.08958,dropout_rate2 = 0.1817,dropout_rate3 = 0.05872,dropout_rate4 = 0.05756, lr = 0.001387)
history = model.fit(train_X,train_Y, epochs=1000, callbacks = [early_stop, ckpt] ,validation_split = 0.1, batch_size=1000, shuffle=True)

Train on 729000 samples, validate on 81000 samples
Epoch 1/1000
729000/729000 [==============================] - 16s 22us/step - loss: 58.3081 - mean_absolute_error: 58.3081 - val_loss: 51.3143 - val_mean_absolute_error: 51.3143
Epoch 2/1000
729000/729000 [==============================] - 14s 19us/step - loss: 33.8116 - mean_absolute_error: 33.8116 - val_loss: 19.2209 - val_mean_absolute_error: 19.2209
Epoch 3/1000
729000/729000 [==============================] - 14s 19us/step - loss: 17.9350 - mean_absolute_error: 17.9350 - val_loss: 10.6981 - val_mean_absolute_error: 10.6981
Epoch 4/1000
729000/729000 [==============================] - 14s 19us/step - loss: 13.9522 - mean_absolute_error: 13.9522 - val_loss: 8.1371 - val_mean_absolute_error: 8.1371
Epoch 5/1000
729000/729000 [==============================] - 14s 19us/step - loss: 12.1152 - mean_absolute_error: 12.1152 - val_loss: 7.6466 - val_mean_absolute_error: 7.6466
Epoch 6/1000
729000/729000 [==============================] - 1

<Figure size 936x576 with 0 Axes>

In [0]:
model.load_weights(ckpt_dir +'/tuning_316_valloss1.68.hdf5')
y1 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_379_valloss1.60.hdf5')
y2 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_445_valloss1.53.hdf5')
y3 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_261_valloss1.78.hdf5')
y4 = model.predict(train_X)
model.load_weights(ckpt_dir +'/tuning_203_valloss1.96.hdf5')
y5 = model.predict(train_X)

In [29]:
frame1, frame2, frame3, frame4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
frame1["model1"] = y1[:,0]
frame1["model2"] = y2[:,0]
frame1["model3"] = y3[:,0]
frame1["model4"] = y4[:,0]
frame1["model5"] = y5[:,0]

frame2["model1"] = y1[:,1]
frame2["model2"] = y2[:,1]
frame2["model3"] = y3[:,1]
frame2["model4"] = y4[:,1]
frame2["model5"] = y5[:,1]

frame3["model1"] = y1[:,2]
frame3["model2"] = y2[:,2]
frame3["model3"] = y3[:,2]
frame3["model4"] = y4[:,2]
frame3["model5"] = y5[:,2]

frame4["model1"] = y1[:,3]
frame4["model2"] = y2[:,3]
frame4["model3"] = y3[:,3]
frame4["model4"] = y4[:,3]
frame4["model5"] = y5[:,3]
!pip install lightgbm

In [41]:
from sklearn.metrics import mean_absolute_error

mean1 = np.mean(frame4,axis=1)
mean_absolute_error(train_Y["layer_4"], mean1)

0.9830110029385414

In [69]:
fr1, fr2, fr3, fr4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
fr1["model1"] = pred_y1[:,0]
fr1["model2"] = pred_y2[:,0]
fr1["model3"] = pred_y3[:,0]
fr1["model4"] = pred_y4[:,0]
fr1["model5"] = pred_y5[:,0]

fr2["model1"] = pred_y1[:,1]
fr2["model2"] = pred_y2[:,1]
fr2["model3"] = pred_y3[:,1]
fr2["model4"] = pred_y4[:,1]
fr2["model5"] = pred_y5[:,1]

fr3["model1"] = pred_y1[:,2]
fr3["model2"] = pred_y2[:,2]
fr3["model3"] = pred_y3[:,2]
fr3["model4"] = pred_y4[:,2]
fr3["model5"] = pred_y5[:,2]

fr4["model1"] = pred_y1[:,3]
fr4["model2"] = pred_y2[:,3]
fr4["model3"] = pred_y3[:,3]
fr4["model4"] = pred_y4[:,3]
fr4["model5"] = pred_y5[:,3]



10000

In [70]:
import lightgbm as lgb
lgb_param = {'boosting_type':'gbdt',
             'num_leaves': 50,
             'max_depth': -1,
            'learning_rate': 0.005, 
            'bagging_fraction' : 1,
            'max_bin' : 5000 ,
            'bagging_freq': 20,
            'colsample_bytree': 0.6,
             'metric': 'mae',
             'min_split_gain': 0.5,
            'min_child_weight': 1,
            'min_child_samples': 20,
            'scale_pos_weight':1,
            'zero_as_missing': True,
            'objective': 'regression', # 목적 함수
            }
# train_set = lgb.Dataset(frame1, train_Y["layer_1"], silent=False)
train_set = lgb.Dataset(frame1.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_1"], silent=False)
valid_set = lgb.Dataset(frame1.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_1"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 5000 ,verbose_eval=50 , valid_sets=valid_set)
predict1 = lgb_model.predict(fr1)

train_set = lgb.Dataset(frame2.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_2"], silent=False)
valid_set = lgb.Dataset(frame2.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_2"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 5000 ,verbose_eval=50, valid_sets=valid_set)
predict2 = lgb_model.predict(fr2)

train_set = lgb.Dataset(frame3.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_3"], silent=False)
valid_set = lgb.Dataset(frame3.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_3"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 5000 ,verbose_eval=50, valid_sets=valid_set)
predict3 = lgb_model.predict(fr3)

train_set = lgb.Dataset(frame4.iloc[:690000,:], train_Y.iloc[:690000,:]["layer_4"], silent=False)
valid_set = lgb.Dataset(frame4.iloc[690000:,:], train_Y.iloc[690000:,:]["layer_4"], silent=False)
lgb_model = lgb.train(params = lgb_param, train_set = train_set , num_boost_round= 5000 ,verbose_eval=50, valid_sets=valid_set)
predict4 = lgb_model.predict(fr4)

[50]	valid_0's l1: 58.3106
[100]	valid_0's l1: 45.3874
[150]	valid_0's l1: 35.3299
[200]	valid_0's l1: 27.5036
[250]	valid_0's l1: 21.4137
[300]	valid_0's l1: 16.6753
[350]	valid_0's l1: 12.9893
[400]	valid_0's l1: 10.123
[450]	valid_0's l1: 7.89464
[500]	valid_0's l1: 6.16387
[550]	valid_0's l1: 4.82173
[600]	valid_0's l1: 3.78199
[650]	valid_0's l1: 2.97658
[700]	valid_0's l1: 2.35339
[750]	valid_0's l1: 1.87167
[800]	valid_0's l1: 1.49954
[850]	valid_0's l1: 1.21203
[900]	valid_0's l1: 0.989912
[950]	valid_0's l1: 0.818373
[1000]	valid_0's l1: 0.68587
[1050]	valid_0's l1: 0.583602
[1100]	valid_0's l1: 0.504478
[1150]	valid_0's l1: 0.443309
[1200]	valid_0's l1: 0.39567
[1250]	valid_0's l1: 0.358716
[1300]	valid_0's l1: 0.330217
[1350]	valid_0's l1: 0.308051
[1400]	valid_0's l1: 0.290846
[1450]	valid_0's l1: 0.277455
[1500]	valid_0's l1: 0.267161
[1550]	valid_0's l1: 0.259182
[1600]	valid_0's l1: 0.252881
[1650]	valid_0's l1: 0.247909
[1700]	valid_0's l1: 0.244021
[1750]	valid_0's l1:

In [0]:
sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1] = predict1
sample.iloc[:,2] = predict2
sample.iloc[:,3] = predict3
sample.iloc[:,4] = predict4

In [0]:
model.load_weights(ckpt_dir +'/tuning_316_valloss1.68.hdf5')
pred_y1 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_379_valloss1.60.hdf5')
pred_y2 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_445_valloss1.53.hdf5')
pred_y3 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_261_valloss1.78.hdf5')
pred_y4 = model.predict(test_X)
model.load_weights(ckpt_dir +'/tuning_203_valloss1.96.hdf5')
pred_y5 = model.predict(test_X)

In [0]:
fr1, fr2, fr3, fr4 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
fr1["model1"] = pred_y1[:,0]
fr1["model2"] = pred_y2[:,0]
fr1["model3"] = pred_y3[:,0]
fr1["model4"] = pred_y4[:,0]
fr1["model5"] = pred_y5[:,0]

fr2["model1"] = pred_y1[:,1]
fr2["model2"] = pred_y2[:,1]
fr2["model3"] = pred_y3[:,1]
fr2["model4"] = pred_y4[:,1]
fr2["model5"] = pred_y5[:,1]

fr3["model1"] = pred_y1[:,2]
fr3["model2"] = pred_y2[:,2]
fr3["model3"] = pred_y3[:,2]
fr3["model4"] = pred_y4[:,2]
fr3["model5"] = pred_y5[:,2]

fr4["model1"] = pred_y1[:,3]
fr4["model2"] = pred_y2[:,3]
fr4["model3"] = pred_y3[:,3]
fr4["model4"] = pred_y4[:,3]
fr4["model5"] = pred_y5[:,3]

sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1] = np.mean(fr1,axis=1)
sample.iloc[:,2] = np.mean(fr2,axis=1)
sample.iloc[:,3] = np.mean(fr3,axis=1)
sample.iloc[:,4] = np.mean(fr4,axis=1)

In [23]:
sample = pd.read_csv('/gdrive/My Drive/Data/sample_submission.csv')
sample.iloc[:,1:] = pred_y
sample.head()

,id,layer_1,layer_2,layer_3,layer_4
0,0,253.438278,228.213654,130.537521,87.765846
1,1,159.621536,126.801979,236.580307,99.874466
2,2,150.084274,182.445572,269.389252,159.521561
3,3,90.494621,227.391785,190.062576,80.387878
4,4,270.253876,292.293335,247.717560,270.892792


In [0]:
sample.to_csv('/gdrive/My Drive/Data/sample5.csv',index=False)